In [ ]:
#Downloading data from API
import requests
import csv

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=TATAPOWER.BSE&outputsize=full&datatype=csv&apikey=VRJ1P0VSUIQ17VCK'
r = requests.get(url)

# Save the CSV data to a file
with open('TATAPOWER.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for line in r.text.splitlines():
        writer.writerow(line.split(','))

print("Data saved to csv")

#initial clean
import pandas as pd
df = pd.read_csv('NTPC.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d', errors='coerce')
df.fillna(df.drop('timestamp', axis=1).mean(), inplace=True)
df.drop_duplicates(inplace=True)
print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())
print(df.duplicated().sum())
df.to_csv('cleaned_TATAPOWER.csv',index=False)


#final clean and adding day_name
import pandas as pd

# Load the dataset
df = pd.read_csv('cleaned_NTPC.csv')

# Display the first few rows and check for missing values
print(df.head())
print("\nBefore cleaning:\n")
print(df.isnull().sum())

# Convert 'timestamp' column to datetime and set as index (assuming there's a timestamp column)
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    # Uncomment the line below if you want to set 'timestamp' as the index
    # df.set_index('timestamp', inplace=True)

# Add 'Days' column
if 'timestamp' in df.columns:
    # df['Days'] = (df['timestamp'] - df['timestamp'].min()).dt.days
    # Add 'Day_Name' column
    df['Day_Name'] = df['timestamp'].dt.day_name()

# Handling missing values
# Fill missing numeric values with the previous value
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].ffill().bfill()

# Fill missing categorical values with the mode
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Verify that there are no more missing values
print("\nAfter cleaning:\n")
print(df.isnull().sum())

# Save the cleaned dataset to a new CSV file
df.to_csv('NTPC_cleaned_day.csv', index=False)